In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg') # workaround, there may be a better way
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats.distributions as dist

In [11]:
da = pd.read_csv("nhanes_2015_2016.csv")

In [12]:
da['SMQ020x'] = da.SMQ020.replace({1:'Yes', 2:'No', 7:np.nan, 9:np.nan})
da['RIAGENDRx'] = da.RIAGENDR.replace({1:'Male', 2:'Female'})
da['DMDCITZNx'] = da.DMDCITZN.replace({1: 'Yes', 2:'No', 7:np.nan, 9:np.nan})

# Hypothesis Testing

In this notebook we demonstrate formal hypothesis testing using the NHANES data.

It is important to note that the NHANES data are a "complex survey".  The data are not an independent and representative sample from the target population.  Proper analysis of complex survey data should make use of additional information about how the data were collected.  Since complex survey analysis is a somewhat specialized topic, we ignore this aspect of the data here, and analyze the NHANES data as if it were an independent and identically distributed sample from a population.

In [13]:
x = da.SMQ020x.dropna() == 'Yes'
p = x.mean()
se = np.sqrt(0.4 * 0.6 / len(x))
test_stat = (p - 0.4) / se
pvalue = 2*dist.norm.cdf(-np.abs(test_stat))
print(test_stat, pvalue)


0.7823563854332805 0.4340051581348052


In [14]:
print(sm.stats.proportions_ztest(x.sum(), len(x), 0.4))
print(sm.stats.proportions_ztest(x.sum(), len(x), 0.4, prop_var = 0.4))
print(sm.stats.binom_test(x.sum(), len(x), 0.4))

(0.7807518954896244, 0.43494843171868214)
(0.7823563854332805, 0.4340051581348052)
0.43403608544100336


/Users/seunghoonyang/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_discrete_distns.py:67: RuntimeWarning: divide by zero encountered in _binom_pdf
  return _boost._binom_pdf(x, n, p)


### Hypothesis Tests for Two Proportions

Comparative tests tend to be used much more frequently than tests comparing one population to a fixed value.  A two-sample test of proportions is used to assess whether the proportion of individuals with some trait differs between two sub-populations.  For example, we can compare the smoking rates between females and males. Since smoking rates vary strongly with age, we do this in the subpopulation of people between 20 and 25 years of age.  In the cell below, we carry out this test without using any libraries, implementing all the test procedures covered elsewhere in the course using Python code.  We find that the smoking rate for men is around 10 percentage points greater than the smoking rate for females, and this difference is statistically significant (the p-value is around 0.01).

In [17]:
dx = da[['SMQ020x', 'RIDAGEYR', 'RIAGENDRx']].dropna()

In [18]:
dx = dx.loc[(dx.RIDAGEYR >= 20) & (dx.RIDAGEYR <= 25), :]

In [19]:
dx

,SMQ020x,RIDAGEYR,RIAGENDRx
6,Yes,22,Male
17,No,24,Female
26,Yes,22,Male
38,No,20,Female
40,Yes,24,Male
...,...,...,...
5688,No,25,Male
5701,No,25,Male
5707,No,25,Female
5729,No,25,Male


In [22]:
p = dx.groupby('RIAGENDRx')['SMQ020x'].agg([lambda z: np.mean(z == "Yes"), "size"])
p.columns = ['Smoke', 'N']
print(p)

              Smoke    N
RIAGENDRx               
Female     0.238971  272
Male       0.341270  252


In [23]:
p_comb = (dx.SMQ020x == 'Yes').mean()
va = p_comb * (1-p_comb)
se = np.sqrt(va * (1 / p.N.Female + 1 / p.N.Male))

In [25]:
test_stat = (p.Smoke.Female - p.Smoke.Male) / se
pvalue = 2*dist.norm.cdf(-np.abs(test_stat))
print(test_stat, pvalue)

-2.5833303066279414 0.009785159057508375


In [27]:
dx_females = dx.loc[dx.RIAGENDRx=='Female', 'SMQ020x'].replace({'Yes':1, 'No':2})
dx_males = dx.loc[dx.RIAGENDRx=='Male', 'SMQ020x'].replace({'Yes':1, 'No':2})
sm.stats.ttest_ind(dx_females, dx_males)

(2.594973144626931, 0.00972590232121263, 522.0)

In [30]:
dx = da[['BPXSY1', 'RIDAGEYR', 'RIAGENDRx']].dropna()
dx = dx.loc[(dx.RIDAGEYR >= 40) & (dx.RIDAGEYR <= 50) & (dx.RIAGENDRx == "Male"), :]
print(dx.BPXSY1.mean())
sm.stats.ztest(dx.BPXSY1, value=120)

125.86698337292161


(7.469764137102597, 8.033869113167905e-14)

In [31]:
dx = da[["BPXSY1", "RIDAGEYR", "RIAGENDRx"]].dropna()
dx = dx.loc[(dx.RIDAGEYR >= 50) & (dx.RIDAGEYR <= 60), :]
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDRx
1,146.0,53,Male
3,132.0,56,Female
9,178.0,56,Male
15,134.0,57,Female
19,136.0,54,Female


In [32]:
bpx_female = dx.loc[dx.RIAGENDRx=="Female", "BPXSY1"]
bpx_male = dx.loc[dx.RIAGENDRx=="Male", "BPXSY1"]
print(bpx_female.mean(), bpx_male.mean())

127.92561983471074 129.23829787234044


In [33]:
print(sm.stats.ztest(bpx_female, bpx_male))
print(sm.stats.ttest_ind(bpx_female, bpx_male))

(-1.105435895556249, 0.2689707570859362)
(-1.105435895556249, 0.26925004137768577, 952.0)


In [34]:
dx = da[["BMXBMI", "RIDAGEYR", "RIAGENDRx"]].dropna()
da["agegrp"] = pd.cut(da.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80])
da.groupby(["agegrp", "RIAGENDRx"])["BMXBMI"].agg(np.std).unstack()

RIAGENDRx,Female,Male
agegrp,,
"(18, 30]",7.745893,6.649440
"(30, 40]",8.315608,6.622412
"(40, 50]",8.076195,6.407076
"(50, 60]",7.575848,5.914373
"(60, 70]",7.604514,5.933307
"(70, 80]",6.284968,4.974855


In [35]:
for k, v in da.groupby("agegrp"):
    bmi_female = v.loc[v.RIAGENDRx=="Female", "BMXBMI"].dropna()
    bmi_female = sm.stats.DescrStatsW(bmi_female)
    bmi_male = v.loc[v.RIAGENDRx=="Male", "BMXBMI"].dropna()
    bmi_male = sm.stats.DescrStatsW(bmi_male)
    print(k)
    print("pooled: ", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar='pooled'))
    print("unequal: ", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar='unequal'))
    print()

(18, 30]
pooled:  (1.7026932933643306, 0.08862548061449803)
unequal:  (1.7174610823927183, 0.08589495934713169)

(30, 40]
pooled:  (1.4378280405644919, 0.15048285114648174)
unequal:  (1.4437869620833497, 0.1487989105789246)

(40, 50]
pooled:  (2.8933761158070186, 0.003811246059501354)
unequal:  (2.9678691663536725, 0.0029987194174035366)

(50, 60]
pooled:  (3.362108779981383, 0.0007734964571391287)
unequal:  (3.3754943901739387, 0.0007368319423226156)

(60, 70]
pooled:  (3.617240144243268, 0.00029776102103194453)
unequal:  (3.628483094544553, 0.00028509141471492935)

(70, 80]
pooled:  (2.926729252512241, 0.003425469414486057)
unequal:  (2.9377798867692064, 0.0033057163315194853)

